# JNMongo で複数ファイルの処理を行う
今回は前回の応用として、複数のファイルを連続的に処理する例を示します。    
今回利用させていただいたのは、
`resources/words.txt` に一覧されています。   
本来はもっと大規模に実験するべきですが、その場合は、IDR の ニコニコ大百科データセットを用いて下さい。

In [1]:
!cat resources/words.txt

エクス・アルビオ.json
コアラ.json
三枝明那.json
鈴原るる.json

## 処理をファイル名を引数にした関数に変える
前回は１ファイルについて処理をしていましたが、今回は複数ファイルを対象にしています。    
なので、これをファイルに対して不変の処理に処理に変える必要があります。

In [2]:
import pandas as pd
from parse import json_parser
from pathlib import Path
from typing import List, Dict
from service import utils, related_words, absts, contents

今回は

1. データを構造体の形式に持ち込む
2. その構造体についてMongoDBへ保存する

という手順で処理を行います。
Pythonで、構造体を定義する場合は辞書やクラスが考えられますが、今回はクラスを用います。

In [3]:
class ParsedArticle:
    def __init__(self, file: Path):
        self.raw_data = json_parser.read_json(file)
        self.title = utils.get_element(self.raw_data, "title")
        self.title_head = utils.get_element(self.raw_data, "title-head")
        self.keywords = utils.get_element(
            self.raw_data, "keywords", 
            message="this article has no keywords")
        self.raw_related_words, self.related_words = related_words.extract_item(self.raw_data)
        self.related_words_len = len(self.related_words)
        self.abstruct, self.sections = self.parse_article()

    def parse_article(self):
        article = utils.get_element(
            self.raw_data, "article", message="this article has no body")
        import copy
        abstruct, _ ,left = absts.extract_abstruct(article)
        sections, _= contents.SectionParser().parse(left)
        return abstruct, sections

クラスが定義できたので、こちらを実験し、前回と同様の処理が行えていることを確認します。

In [4]:
sample_art = ParsedArticle(Path("./resources/三枝明那.json"))

In [6]:
def view_article(art: ParsedArticle):
    data = {}
    data["title"] = art.title
    data["title-head"] = art.title_head
    data["related-words"] = art.related_words
    data["related-words_raw"] = art.raw_related_words
    data["#related-words"] = art.related_words_len
    data["keywords"] = art.keywords
    data["abstruct"] = str(art.abstruct)
    data["section_keys"] = [str(sec.titles) for sec in art.sections]
    data["section_sample(key)"] = art.sections[0].titles
    data["section_sample(value)"] = art.sections[0].contents
    return pd.DataFrame(data.values(), index=data.keys(), columns=["value"])
view_article(sample_art)

,value
title,エクス・アルビオ
title-head,エクス・アルビオとは (エクスアルビオとは) [単語記事] - ニコニコ大百科
related-words,"[三枝明那（ウザ絡みしてくる先輩）, 🏰御伽原江良, バーチャルYouTuber, ♨️小野..."
related-words_raw,"[[{'content': ['バーチャルYouTuber', 'にじさんじ', 'いちから..."
#related-words,30
keywords,"[エクス・アルビオ, エクスアルビオ, にじさんじ, クズ, コラボ, バー, 英雄, ゲーム]"
abstruct,[['エクス・アルビオとは、いちから株式会社が運営する「にじさんじ」所属のバーチャルライバー...
section_keys,"[('関連リンク',), ('関連項目',)]"
section_sample(key),"(関連リンク,)"
section_sample(value),"({'content': [{'content': ['にじさんじ公式の紹介'], 'tag..."


# 

他のファイルでも実行してみましょう。

In [7]:
sample_art = ParsedArticle(Path("./resources/エクス・アルビオ.json"))
view_article(sample_art)

,value
title,エクス・アルビオ
title-head,エクス・アルビオとは (エクスアルビオとは) [単語記事] - ニコニコ大百科
related-words,"[三枝明那（ウザ絡みしてくる先輩）, 🏰御伽原江良, バーチャルYouTuber, ♨️小野..."
related-words_raw,"[[{'content': ['バーチャルYouTuber', 'にじさんじ', 'いちから..."
#related-words,30
keywords,"[エクス・アルビオ, エクスアルビオ, にじさんじ, クズ, コラボ, バー, 英雄, ゲーム]"
abstruct,[['エクス・アルビオとは、いちから株式会社が運営する「にじさんじ」所属のバーチャルライバー...
section_keys,"[('関連リンク',), ('関連項目',)]"
section_sample(key),"(関連リンク,)"
section_sample(value),"({'content': [{'content': ['にじさんじ公式の紹介'], 'tag..."


In [8]:
sample_art = ParsedArticle(Path("./resources/コアラ.json"))
view_article(sample_art)

,value
title,コアラ
title-head,コアラとは (コアラとは) [単語記事] - ニコニコ大百科
related-words,"[コアラ(けものフレンズ), 多摩動物公園： 飼育・公開中（1頭のみ。2017年5月現在）,..."
related-words_raw,"[[{'content': ['コアラのマーチ', 'オーストラリア', '動物', {'c..."
#related-words,8
keywords,"[コアラ, オーストラリア, 動物園, 動物, 名古屋市, 子供, マスコット]"
abstruct,"[['コアラとは、'], {'content': [{'content': ['オーストラリ..."
section_keys,"[('関連項目',), ('関連リンク',), ('脚注',)]"
section_sample(key),"(関連項目,)"
section_sample(value),"({'content': [{'content': ['コアラのマーチ'], 'tag': ..."


In [12]:
sample_art = ParsedArticle(Path("./resources/鈴原るる.json"))
view_article(sample_art)

,value
title,鈴原るる
title-head,鈴原るるとは (スズハラルルとは) [単語記事] - ニコニコ大百科
related-words,"[🏰御伽原江良, バーチャルYouTuber, 🔲レヴィ・エリファ, 🌖夢月ロア, 🧠久遠千..."
related-words_raw,"[[{'content': ['にじさんじ', 'いちから株式会社', 'バーチャルYouT..."
#related-words,23
keywords,"[鈴原るる, スズハラルル, コラボ, でびでび・でびる, にじさんじ, バーチャルライバー..."
abstruct,"[['（・w・🎀）'], ['鈴原るる（すずはら-）とは、いちから株式会社が運営する「にじさ..."
section_keys,"[('実況配信したゲーム', 'レトロゲーム'), ('実況配信したゲーム', 'ホラーゲー..."
section_sample(key),"(実況配信したゲーム, レトロゲーム)"
section_sample(value),"({'content': [{'content': ['超魔界村'], 'tag': 'li..."
